# Variational Quantum Algorithm

<div style="text-align: center;">
<img src="/Users/nrd/quantum/QPlay/algos/imgs/Screenshot 2025-10-28 at 12.21.02 PM.png" width="400">
</div>

The idea behind VQA's are that some of the rotational values in the circuit are parametric

### Simple Definition

VQA's are circuits that use parametric values for $\theta$ in rotations


## Types

### Quantum Approximation Optimization Algorithm

To best understand QAOA, we must first understand the concept of **max-cut**

**Max-Cut:** Given a graph $G(V, E)$, split $V$ into two subsets $S$ and $T$ such that the sum of edge weights between $S$ and $T$ is maximized

In [1]:
# Qiskit 1.x style
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator, Sampler
from qiskit_aer.primitives import Estimator as AerEstimator, Sampler as AerSampler
import numpy as np

# ----- 1) Problem: square graph -----
n = 4
edges = [(0,1),(1,2),(2,3),(3,0)]

# Build H_C = sum_{(i,j)} (I - ZiZj)/2  as a SparsePauliOp
terms = []
coeffs = []
for (i,j) in edges:
    z_string = ['I']*n
    z_string[i] = 'Z'
    z_string[j] = 'Z'
    terms.append(''.join(reversed(z_string)))  # Qiskit endianness
    coeffs.append(-0.5)  # -ZiZj/2
# Add +|E|/2 * I
Hc = SparsePauliOp.from_list([(p, c) for p,c in zip(terms, coeffs)])
Hc = Hc + SparsePauliOp.from_list([('I'*n, len(edges)/2)])

# ----- 2) QAOA circuit builder for given (gammas, betas) -----
def qaoa_circuit(gammas, betas):
    p = len(gammas)
    qc = QuantumCircuit(n)
    # |+>^n
    for q in range(n):
        qc.h(q)
    for k in range(p):
        gamma = gammas[k]
        beta  = betas[k]
        # Cost unitary
        for (i,j) in edges:
            qc.cx(i, j)
            qc.rz(2*gamma, j)
            qc.cx(i, j)
        # Mixer unitary
        for q in range(n):
            qc.rx(2*beta, q)
    return qc

# ----- 3) Objective: expected cut value -----
estimator = AerEstimator()  # fast local simulator
def expected_cut(params):
    p = len(params)//2
    gammas = params[:p]
    betas  = params[p:]
    qc = qaoa_circuit(gammas, betas)
    # We *maximize* expected cut; optimizers minimize -> return negative
    value = estimator.run([(qc, Hc)]).result().values[0]
    return -value

# ----- 4) Optimize (p = 1 to keep it tiny) -----
p = 1
# heuristic init: gammas in [0, pi], betas in [0, pi/2]
x0 = np.array([0.8, 0.6])  # [gamma_1, beta_1]

from scipy.optimize import minimize
res = minimize(expected_cut, x0, method="Nelder-Mead",
               options={"maxiter": 100, "xatol": 1e-3, "fatol": 1e-3})

best_gammas = res.x[:p]
best_betas  = res.x[p:]
best_exp_cut = -res.fun  # negate back

print("p =", p)
print("gammas:", best_gammas, "betas:", best_betas)
print("Expected cut value (approx.):", best_exp_cut)

# ----- 5) Sample a concrete solution -----
sampler = AerSampler()
qc_best = qaoa_circuit(best_gammas, best_betas)
qc_best.measure_all()
shots = 5000
dist = sampler.run(qc_best, shots=shots).result().quasi_dists[0]

# score bitstrings and pick the best
def cut_value(bitstr):
    # bitstr is little-endian from Qiskit; flip for intuitive indexing
    b = bitstr[::-1]
    val = 0
    for (i,j) in edges:
        val += (b[i] != b[j])
    return val

best_b, best_score = None, -1
for b, p in dist.items():
    # convert int to n-bit string
    s = format(b, f'0{n}b')
    sc = cut_value(s)
    if sc > best_score:
        best_score, best_b = sc, s

print("Best sampled bitstring:", best_b, "score:", best_score, "out of", len(edges))


ImportError: cannot import name 'Estimator' from 'qiskit.primitives' (/Users/nrd/miniconda3/envs/qk/lib/python3.13/site-packages/qiskit/primitives/__init__.py)